Comparer le LS avec régression linéaire et le LS lorsqu'on utilise un réseau de neurones (régression non-linéaire) pour apprendre la fonction de continuation. 

C'est un travail numérique prospectif à faire pour le mardi 11 mars en s'interrogeant sur les différents paramètres: nombres de scénarios, nombre d'epochs, taille de batch, réglage du learning rate, normalisation des données simulées...

In [5]:
import torch
from torch import nn 
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import numpy as np 

nombres de scénarios, nombre d'epochs, taille de batch, réglage du learning rate, normalisation des données simulées...

la fonction de continuation est une focntion de R dans R ,  #8 neuronnes 4 neuronnes 2 couches  8 neuronnes equivalents à (16+ 8*4+4 +4)

#MSloss pour les probleme de moindre carre 
#Relu
#linear pour une couche d'appli linéaire 


#il ya 2 syntaxe (celle du prof est deja direct mais il y en a une autre)

#Feedforward Neural Network :(je sais pas pourquoi ce mot technique )

#la question est: si j 'ai une fonction avec une brisure comme c'est le cas ici, qu'elle est la bonne architecture pour detectee cettte application (presquelinéaire) par morceau .

#peut etre ajouter une fonction sa'activation à la fin pour avoir que la partie positif?

#loss.backward calcule le loss parapoort à theata 

#full c'est quan don considere l'esperance

#SGD c'est quand on considere un seul path?


In [18]:
# choisir un optimiseur , adam ou 

np.random.seed(42)  # fix the seed for every valuation

In [19]:
#paramétres
r=0.1
sigma=0.25
x_0=100
K=110
N=10   
T=1
npaths=1000000
ts = np.linspace(0, T, N+1)

In [20]:
# Simulation Monte-Carlo du processus Black-Scholes

def blackscholes_mc(S, vol, r, ts, npaths):

    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths)),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r)*ts)
    return paths

In [22]:
paths=blackscholes_mc(x_0, sigma, r, ts, npaths)
paths_torch = torch.from_numpy(paths).float()
print(paths_torch.shape) 

torch.Size([11, 1000000])


In [ ]:
#normalizing

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(  #champs de ces objets?, ou est le biais??)# par defaut c'es true
            nn.Linear(3, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):  #surcharge methode forward, programation fonctionnel?
        out = self.linear_relu_stack(x)
        return out

In [ ]:
Phi = NeuralNetwork()

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

Phi.apply(init_weights)

In [15]:
x = torch.tensor([0.2, 0.3, 0.4])
print(Phi(x))

tensor([0.4451], grad_fn=<ViewBackward0>)


In [12]:
Phi.get_parameter("linear_relu_stack.0.weight")

Parameter containing:
tensor([[-0.4460,  0.3497, -0.2720],
        [-0.2360, -0.5141, -0.4507],
        [-0.2608, -0.4075, -0.2435],
        [ 0.1422, -0.2414, -0.0155],
        [-0.3481,  0.2464,  0.0577],
        [-0.2629, -0.2844, -0.0531],
        [ 0.4364, -0.3653, -0.4121],
        [-0.1599,  0.4968,  0.2690]], requires_grad=True)

In [8]:
Phi(3)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not int

In [ ]:
payoff = np.maximum(K - paths[-1], 0).copy()

# Matrice pour stocker les coefficients du polynôme
poly_coeffs = np.zeros((len(ts) - 1, 4)) 


for i in range(len(ts) - 2, 0, -1):
    discount = np.exp(-r * (ts[i + 1] - ts[i]))
    payoff *= discount
    
    # Régression polynomiale
    p = np.polyfit(paths[i], payoff, deg=3) #polynôme de degré 3
    poly_coeffs[i] = p  
    contval = np.polyval(p, paths[i])  # Valeur de continuation
    exerval = np.maximum(K - paths[i], 0)  # Valeur d'exercice
    
    C[i] = contval
    V[i] = np.maximum(exerval, contval)
    
    # Identifier les chemins où il faut exercer
    ind = exerval > contval
    payoff[ind] = exerval[ind]


print("Valeur de l'option:",max(np.mean(payoff*np.exp(-r*(ts[1]-ts[0]))),10)) # K-x_0=10